# --- Day 9: Movie Theater ---


In [1]:
# --- Support Functions ---
# This section contains support functions used by the main code.

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

def get_area(a:tuple[int,int], b:tuple[int,int]) -> int:
    s1 = abs(a[0] - b[0]) + 1
    s2 = abs(a[1] - b[1]) + 1
    return s1 * s2


In [2]:
# --- Part ONE ---

input = read_input('input.txt')

tiles = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

# print (tiles)

areas = []
for i, a in enumerate(tiles):
    for b in tiles[i+1:]:
        areas.append(get_area(a, b))

#print (areas)
result = max(areas)


print("Part ONE:", result)

Part ONE: 4763040296


In [3]:
# Support Functions for Part2

from typing import List, Tuple, Dict, Set

r2c: Dict[Dict[int,int], Dict[int,int]] = {'x':{},'y':{}} # real to compressed coordinates
c2r: Dict[Dict[int,int], Dict[int,int]] = {'x':{},'y':{}} # compressed to real coordinates

def compress_grid(corners: List[Tuple[int,int]]) -> List[Tuple[int,int]]:
    """ Remap the grid to use only used coordinates and map real
        coordinate in 
    """
    allx = set()
    ally = set()

    for x,y in corners:
        allx.add(x)
        ally.add(y)
    # eventually add a y value between miny and maxy for the minx position
    # needed to bee sure to have a start point for fill_borders
    minx = min(x for x, _ in corners)
    miny = min(y for x, y in corners if x==minx)
    ally.add(y+1)
    allx = sorted(list(allx))
    ally = sorted(list(ally))

    # convert grid and create r2c and c2r for conversion
    compressed_grid: List[Tuple[int,int]] = []
    for x, y in corners:
        nx = allx.index(x)
        ny = ally.index(y)
        r2c['x'][x] = nx
        r2c['y'][y] = ny
        c2r['x'][nx] = x
        c2r['y'][ny] = y
        compressed_grid.append((nx,ny))
    
    return compressed_grid

def create_borders(corners: List[Tuple[int,int]]) -> List[Tuple[int,int]]:
    corners = corners.copy() 
    corners.append(corners[0])
    borders : List[Tuple[int,int]] = []
    for i in range(len(corners)-1):
        p1 = corners[i]
        p2 = corners[i+1]
        p1, p2 = sorted([p1,p2])
        for x in range(p1[0], p2[0]+1):
            for y in range(p1[1], p2[1]+1):
                borders.append((x,y))
    return borders

def fill_borders(borders: List[Tuple[int,int]]) -> Set[Tuple[int,int]]:
    filled: Set[Tuple[int,int]] = set()
    # find the left-top corner
    minx = min(x for x, _ in borders)
    miny = min(y for x, y in borders if x==minx)

    # First internal point
    p0 = (minx+1, miny+1)

    if p0 in borders:
        print(" !!!! ERROR !!!! fill_borders : firse internal point error")
        return filled

    # fill the spaces
    directions = [(-1 ,0), (1, 0), (0, -1), (0, +1)]
    queue: List[Tuple[int,int]] = []
    queue.append(p0)

    while queue:
        p=queue.pop(0)
        if p in borders:
            continue
        filled.add(p)
        for np in [(p[0]+dx, p[1]+dy) for dx,dy in directions]:
            if np in filled or np in queue or np in borders:
                continue
            queue.append(np)
    return set(list(filled) + borders)

def all_compressed_rects(corners: List[Tuple[int,int]]) -> List[Tuple[int,Tuple[int,int],Tuple[int,int]]]:
    allrect = [
            ((abs(x1-x2)+1)*(abs(y1-y2)+1), (r2c['x'][x1], r2c['y'][y1]), (r2c['x'][x2], r2c['y'][y2]))
            for (idx,(x1,y1)) in enumerate(corners[:-1])
            for (x2,y2) in corners[idx+1:]
        ]
    return allrect

def is_rect_inside(p1: Tuple[int,int], p2: Tuple[int,int], grid: Set[Tuple[int,int]]) -> bool:
    x1, y1 = p1
    x2, y2 = p2
    if (x1, y2) not in grid:
        return False
    if (x2, y1) not in grid:
        return False
    x1, x2 = sorted([x1,x2])
    y1, y2 = sorted([y1,y2])
    # fast check

    area = ((x,y) for x in range(x1, x2+1) for y in range(y1, y2+1))
    # print(area)
    inside = all((x,y) in grid for (x,y) in area)
    return inside

def draw_grid(pts: List[Tuple[int,int]], char: str = '#', bg_char: str = '.'):
    """ 
    Print the grid with 'char' for each  coordinate in List and 
    'bg_char' on all other positions
    """
    maxx = max(x for x, _ in pts)
    maxy = max(y for _, y in pts)

    for y in range(maxy+1):
        line = [char if (x,y) in pts else bg_char for x in range(maxx+1)]
        print(''.join(line))



In [4]:
# --- Part TWO ---
input = read_input('input.txt')

result = 0

# Initial tiles are the corner of the perimeter
corners = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

# print ("-- Original Corners --")
# draw_grid(corners)

# print ("-- Original Borders --")
# borders = create_borders(corners)
# draw_grid(borders)

# print ("-- Original Filled --")
# filled = fill_borders(borders)
# draw_grid(filled)


#print ("-- Compressed Corners --")
c_corners = compress_grid(corners)
# draw_grid(c_corners)

#print ("-- Compressed Borders --")
c_borders = create_borders(c_corners)
# draw_grid(c_borders)

#print ("-- Compressed Filled --")
c_filled = fill_borders(c_borders)
# draw_grid(c_filled)

all_rect = sorted(all_compressed_rects(corners),reverse = True)

#for rect in all_rect:
#    print (rect)


for d, p1, p2 in all_rect:
    is_inside = is_rect_inside(p1, p2, c_filled)
    # print(is_inside)
    if is_inside:
        break

result = d
print("Part TWO:", result)

Part TWO: 1396494456
